In [3]:
import json

import bz2
import re
from tqdm import tqdm
from scipy import sparse

In [4]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from pymystem3 import Mystem
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from nltk import FreqDist
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import  ward, dendrogram
from gensim import corpora, models
import gensim
from gensim.models.doc2vec import *
from sklearn.pipeline import Pipeline

Populating the interactive namespace from numpy and matplotlib


In [6]:
responses = []
with bz2.BZ2File('../3/banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:38, 2040.56it/s]



Домашнее задание по NLP 
Составление словарей для классификации по тональности

При классификации текстов или предложений по тональности необходимо использовать оценочные словари для предметной области, то есть, такие словари, в которых содержатся отрицательные и позитивные слова для какой-то предметной области. Идея подобных словарей основана на следующих наблюдениях: во-первых, для разных товаров используются разные оценочные слова (например бывает “захватывающая книга”, но не бывает “захватывающих лыж”), во-вторых, в контексте разных товаров одни и те же слова могут иметь разную окраску (слово “тормоз” в отзыве на велосипед имеет нейтральную окраску, в отзыве на компьютер – резко негативную, “пыль” в контексте пылесосов – нейтральную, в контексте кофемолок – положительную (“мелкий помол в пыль”)). Еще один пример: "теплое пиво" – это плохо, а "теплый свитер" – это хорошо.

Составление таких словарей в ручную – трудоемкий процесс, но, к счастью, его не сложно автоматизировать, если собрать достаточно большие корпуса отзывов. В этом домашнем задании вам предстоит попробовать реализовать один их подходов к составлению оценочных словарей, основанный на статье Inducing Domain-Specific Sentiment Lexicons from Unlabeled Corpora (https://nlp.stanford.edu/pubs/hamilton2016inducing.pdf).

Данные для задания – уже знакомые вам отзывы на банки, собранные с нескольких сайтов Рунета. Отзывы могут быть как положительными (оценка 5), так и отрицательными (оценка 1).

Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова.
Обучите по коллекции предложений word2vec
Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:

тест на определение ближайших слов
тест на аналогии (мужчина – король : женщина – королева)
тест на определение лишнего слова.
Постройте несколько визуализаций:

TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
задайте координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах. Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.

In [7]:
responses[99]

{'city': 'г. Саратов',
 'rating_not_checked': False,
 'title': 'Карта ко вкладу',
 'num_comments': 0,
 'bank_license': 'лицензия № 880',
 'author': 'ronnichka',
 'bank_name': 'Югра',
 'datetime': '2015-06-03 20:56:57',
 'text': 'Здравствуйте! Хотела написать, что мне месяц не выдают карту ко вкладу, ссылаясь на "нам же их из Самары везут" (на секундочку 5 часов езды от нашего города). Но! Прочитала, что людям 3,5 месяцев не выдают карту, и поняла, что у меня все хорошо, пока что. И подарок мне дали, и кулер в отделении есть. Так что я, конечно, готова ждать. Правда хотелось бы не очень долго.',
 'rating_grade': 3}

In [8]:
texts = []
for i in range(len(responses)):
    texts.append(responses[i]['text'])

In [9]:
regex = re.compile("[А-Яа-я]+")
def words_only(text, regex=regex):
    return " ".join(regex.findall(text))


mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

    
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return [l for l in m.lemmatize(text) if l.isalpha()]
    except:
        return []

def preprocessing(text):
    words = words_only(text)
    no_stopwords = remove_stopwords(words)
    lemmas = lemmatize(no_stopwords)
    return(lemmas)

In [11]:
df = pd.DataFrame(texts)

In [12]:
df.columns = ['text']

In [13]:
%time df['lemmas'] = df.text.apply(lambda x: preprocessing(x))

CPU times: user 3min 3s, sys: 9.57 s, total: 3min 12s
Wall time: 21min 53s


In [14]:
%%time

model = Word2Vec(df.lemmas, size=100, window=3, min_count=20, workers=4)

CPU times: user 3min 49s, sys: 1.57 s, total: 3min 51s
Wall time: 1min 11s


In [15]:
model.save("word2v.model")
model = Word2Vec.load("word2v.model")

In [20]:
model.most_similar("машина")

[('автомобиль', 0.8500747680664062),
 ('авто', 0.8358133435249329),
 ('холодильник', 0.7717180252075195),
 ('жилье', 0.7261992692947388),
 ('кровать', 0.7076298594474792),
 ('ноутбук', 0.7044014930725098),
 ('фотоаппарат', 0.6976034045219421),
 ('мебель', 0.695694088935852),
 ('квартира', 0.6892110705375671),
 ('ноут', 0.6802246570587158)]

In [ ]:
# Модель word2vec хорошо оперделила близкие слова к слову "машина"

In [25]:
model.most_similar(positive=["королева", "женщина"], negative=["мужчина"])

[('королев', 0.8208070993423462),
 ('лобня', 0.8204281330108643),
 ('балашиха', 0.8143842816352844),
 ('чехов', 0.8094383478164673),
 ('реутов', 0.8031971454620361),
 ('щелково', 0.8028858304023743),
 ('серпухов', 0.7967113256454468),
 ('раменское', 0.7965644001960754),
 ('волжский', 0.7948329448699951),
 ('йошкар', 0.7917598485946655)]

In [ ]:
# аналогии по корпусу слов определяет. Король прямо не получилось но полуился королев) 

In [28]:
model.doesnt_match("москва питер омск банк".split())

'банк'

In [ ]:
# аналогичным образом хорошо в модели работает тест на определение лишнего слова

In [29]:
top_words = []

fd = FreqDist()
for text in tqdm(df.lemmas):
    fd.update(text)
for i in fd.most_common(500):
    top_words.append(i[0])
print(top_words[:15])

100%|██████████| 153499/153499 [00:19<00:00, 8066.98it/s] 

['банк', 'карта', 'я', 'деньги', 'день', 'мой', 'в', 'кредит', 'который', 'отделение', 'клиент', 'сотрудник', 'счет', 'сказать', 'сумма']


In [30]:
top_words = [w for w in top_words if len(w) > 4]
len(top_words)

398

In [31]:
top_words_vec = model[top_words]

In [32]:
tsne = TSNE(n_components=2, random_state=0)

%time top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 2.7 s, sys: 15.2 ms, total: 2.71 s
Wall time: 2.55 s


In [33]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

In [34]:
pca = PCA(n_components=2)
%time pca_words = pca.fit_transform(top_words_vec)

CPU times: user 63.5 ms, sys: 19 ms, total: 82.5 ms
Wall time: 43.6 ms


In [35]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec PCA for most common words")

source = ColumnDataSource(data=dict(x1=pca_words[:,0],
                                    x2=pca_words[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

In [36]:
horminploho = model['хорошо'] - model['плохо']


In [37]:

bistrominmedlenno = model['быстро'] - model['медленно']

In [38]:
sberx = horminploho*model['сбербанк']

In [39]:
sbery = bistrominmedlenno*model['сбербанк']

In [40]:

sum(sberx)

8.46986

In [41]:
sum(sbery)

10.695286

In [42]:

banki = \
[
  'сбербанк',
  'альфа',
  'втб',
  'хоум',
  'тинькофф',
  'траст',
  'ренессанс',
  'связной', 
  'отп'
]

In [43]:
bank_coord = []
for i in banki:
    x = sum(horminploho*model[i])
    y = sum(bistrominmedlenno*model[i])
    bank_coord.append([i, x, y])

In [44]:
bank_coord

[['сбербанк', 8.46986, 10.695286],
 ['альфа', -4.2594013, 13.356907],
 ['втб', -9.532849, 6.6314034],
 ['хоум', -11.538108, 14.592283],
 ['тинькофф', -4.0065637, 19.322414],
 ['траст', -4.0028934, -4.0829177],
 ['ренессанс', -3.369492, 22.249819],
 ['связной', 3.765994, 18.834852],
 ['отп', -0.9040401, -0.048764467]]

In [45]:
names = []
x = [] 
y = [] 
for i in bank_coord:
    names.append(i[0])
    x.append(i[1])
    y.append(i[2])

In [46]:

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec coords of banks on axis good-bad, fast-slow")

source = ColumnDataSource(data=dict(x1=x,
                                    x2=y,
                                    names=names))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)